 # <center>Итоговый проект "Определение социально уязвимых групп населения" #

 ## Часть 1. Подготовка данных. ##

### <center> 1. Описание данных ###
#### Проектный анализ будет проводиться на предоставленных и загруженных из открытых источников интернета данных: ####

  ➔ **child_mortality_rural_1990_2021.xls** — число умерших на первом году жизни детей за год, по всем регионам,в сельской местности;

  ➔ **child_mortality_urban_1990_2021.xls** — число умерших на первом году жизни детей за год, по всем регионам,в городской местности;
  
  ➔ **disabled_total_by_age_2017_2022.csv** — число людей с инвалидностью по регионам, по месяцам, по возрастным группам;

  ➔ **morbidity_2005_2020_age_disease.xls** — заболеваемость на 100тыс. человек населения, по возрастным группам и группам заболеваний;

  ➔ **poverty_percent_by_regions_1992_2020.csv** — процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам;

  ➔ **welfare_expense_share_2015_2020** — расходы на социальную политику от общих расходов бюджета региона, % в год;

  ➔ **cash_real_income_wages_2015_2020** — среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам;

  ➔ **poverty_socdem_2017-2020.xls** — распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам;

  ➔ **housing_2020.xlsx** — характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование 2020 года;

  ➔ **population.xlsx** — численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг;

  ➔ **gross_regional_product_1996_2020.xls** — валовой региональный продукт на душу населения, в рублях;

  ➔ **regional_production_2016-2020.xlsx** — объём регионального производства или работ/услуг, выполненных собственными силами, 2016–2020 гг. (в млн.руб., значение показателя за год, полный круг)*;

  ➔ **retail_turnover_per_capita_2000_2021.xls** — оборот розничной торговли на душу населения, в рублях;

  ➔ **area_region_rf.xlsx** — площадь регионов РФ, в кв.км *;

  ➔ **trud_15-72.xlsx** — уровень безработицы регионов РФ за период 2000-2021гг. в % *;

  ➔ **newborn_2006_2022_monthly.csv** — рождаемость (за год) за период 2006-2022гг.*;

  ➔ **number of diseases.xls** — среднее число зарегистрированных заболеваний у пациентов с диагнозом, установленном впервые в жизни (без данных Минздрава России), на конец года за период 2011-2021гг.*;

  ➔ **respiratory_diseases.xlsx** — среднее число респираторных заболеваний (на 1тыс человек) за период 2016-2020гг.*;

  ➔ **neoplasms.xlsx** — среднее число онкологических заболеваний (на 1тыс человек) за период 2016-2020гг.*;

  ➔ **disabled_total.xlsx** — число людей с инвалидностью по регионам за период 2017-2022гг.*.

  ➔ Папка **crimes** — сведения опреступлениях, совершённых отдельными категориями лиц за 2016–2022гг., по месяцам, регионам, категориям лиц, категориям преступлений:
   1. В папке содержатся файлы о расследованных преступлениях (число преступлений);
   2. Файлы представлены в том виде, в котором их предоставляет Генпрокуратура.

 
    

**Импортируем нужные библиотеки.**  

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import openpyxl
from pandas.io.excel import ExcelWriter

import regex as re

import pickle

from tools.prepare import fix_name_reg
from tools.columns import add_col

import json

### <center> 2. Предварительный анализ и подготовка данных

Для преобразования и отчистки данных воспользуемся созданными инструментамми tools.prepare.

In [2]:
# загрузим в кэш словарь регулярных значений названий регионов РФ из pickle-файла
with open('tools/region_names.pkl', 'rb') as n:
        region_names = pickle.load(n)

#### 2.1. Численность населения по субъектам РФ за период 1999-2022гг., чел. 

In [3]:
population = pd.read_excel(
        'social_russia_data/population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)

names_to_drop = [
        'Архангельская область (кроме Ненецкого автономного округа)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
        'Муниципальные образования Алтайского края',
        'Алтайский муниципальный район',
        'Алтайский сельсовет',
        'Большекалтайский сельсовет',
        'Населенные пункты, входящие в состав муниципальных образований Алтайского края',
        'Кировский сельсовет'
]
population = fix_name_reg(population, region_names, names_to_drop)
population.loc['Архангельская область без Ненецкого АО'] \
    = (population.loc['Архангельская область без Ненецкого АО']
      - population.loc['Ненецкий автономный округ'])
population.loc['Тюменская область без округов'] \
    = (population.loc['Тюменская область без округов']
       - population.loc['Ханты-Мансийский автономный округ - Югра']
       - population.loc['Ямало-Ненецкий автономный округ'])

newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = pd.to_datetime(newcols).year

display('Численность населения региона на 1 января года, чел.', population.head(3))
display('Колличество регионов - %.f' % population.shape[0])


'Численность населения региона на 1 января года, чел.'

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2662738.0,2651628.0,2641079.0,2621050.0,2602595.0,2571987.0,2539430.0,2503510.0,2473024.0,2453455.0,...,2398751.0,2390638.0,2384812.0,2376774.0,2365680.0,2350080.0,2332813.0,2317153.0,2296353.0,2268179.0
Амурская область,949526.0,935607.0,923055.0,911381.0,901044.0,887781.0,874018.0,861056.0,850502.0,844290.0,...,816910.0,811274.0,809873.0,805689.0,801752.0,798424.0,793194.0,790044.0,781846.0,772525.0
Архангельская область без Ненецкого АО,1372631.0,1349160.0,1328187.0,1309552.0,1290956.0,1273668.0,1257312.0,1239924.0,1224813.0,1215264.0,...,1159506.0,1148760.0,1139950.0,1130240.0,1121813.0,1111031.0,1100290.0,1092424.0,1082662.0,1069782.0


'Колличество регионов - 85'

#### 2.2. Площадь субъекта РФ, кв.км ####

Данные взяты с [https://ru.wikipedia.org](https://ru.wikipedia.org/w/index.php?title=%D0%9F%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8&action=edit)

In [4]:
# считаем данные площади регионов с excel-листа 
area = pd.read_excel('social_russia_data/area_region_rf.xlsx')
names_to_drop = ['Архангельская область включая НАО', 'Тюменская область без ХМАО и ЯНАО']
area = fix_name_reg(area, region_names, names_to_drop)

display('Площадь субьекта РФ, кв.км', area.head(3))
display('Колличество регионов - %.f' % area.shape[0])

'Площадь субьекта РФ, кв.км'

,area
region,
Алтайский край,167996
Амурская область,361908
Архангельская область без Ненецкого АО,413103


'Колличество регионов - 85'

#### 2.3. Жилищные условия в регионах по данным 2020 г. ####


* **living area** - средняя жилая площадь на одного человека.

In [5]:
def get_housing(file, sheet, cols, skipfooter=0):
    "Считывание данных с excel-листа"
    df = pd.read_excel(
        file, sheet_name=sheet, usecols=cols, skipfooter=skipfooter)
    df.rename({'Регион': 'region'}, axis=1, inplace=True)
    
    # Ошибки в написании названий (латиница?)
    df.replace({'Bладимирская область': 'Владимирская'}, inplace=True)
    df.replace({'Bолгоградская область': 'Волгоградская'}, inplace=True)
    df.replace({'Bологодская область': 'Вологодская'}, inplace=True)
    df.replace({'Bоронежская область': 'Воронежская'}, inplace=True)
    df.replace('...', 0, inplace=True)  # слабая статистика -> 0
    df = fix_name_reg(df, region_names)
    return df

file = 'social_russia_data/housing_2020.xlsx'

living_area = get_housing(file, 0, 'A, H')
living_area.columns.values[0] = 'living area'
living_area.rename(columns={'living area':'2020'},inplace=True)
living_area.columns = pd.to_datetime(living_area.columns, format='%Y').year

# выведем значения за 2020гг.
display('Жилищные условия в регионах.', living_area.head(3))
display('Колличество регионов - %.f' % living_area.shape[0])

'Жилищные условия в регионах.'

,2020
region,
Алтайский край,18.0
Амурская область,17.3
Архангельская область без Ненецкого АО,16.9


'Колличество регионов - 85'

#### 2.4. Доходы населения в рублях за 2015-2020гг.

In [6]:
def finances(file, sheet):    
    # Считывание финансовых данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = fix_name_reg(df, region_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

file = './social_russia_data/cash_real_income_wages_2015_2020.xlsx'

per_capita = finances(file, 0) # Среднедушевые денежные доходы (в месяц), руб.
real_incomes = finances(file, 1) # Реальные денежные доходы, в процентах к предыдущему году (покупательная способность: с учетом роста цен)
formal_wage = finances(file, 2) # Среднемесячная номинальная начисленная заработная плата, руб.
real_pay = finances(file, 3) # Реальная начисленная заработная плата, в процентах к предыдущему году (покупательная способность: с учетом роста цен)

display('Среднедушевые денежные доходы (в месяц), руб.', per_capita.head(3))
display('Среднемесячная номинальная начисленная заработная плата , руб.', formal_wage.head(3))
display('Колличество регионов - %.f' % per_capita.shape[0])

'Среднедушевые денежные доходы (в месяц), руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20860.0,21256.0,22139.0,22829.0,23937.0,23864.0
Амурская область,28240.0,27976.0,29213.0,30937.0,33304.0,35499.0
Архангельская область без Ненецкого АО,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0


'Среднемесячная номинальная начисленная заработная плата , руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20090.0,21202.0,22743.0,25519.0,27962.0,30072.0
Амурская область,32902.0,33837.0,37368.0,42315.0,47234.0,52430.0
Архангельская область без Ненецкого АО,35592.0,38118.0,40352.0,45427.0,49435.0,52779.0


'Колличество регионов - 85'

#### 2.5. Валовой региональный продукт на душу населения за период 2011-2020гг., руб.

In [7]:
gdpp = pd.read_excel('social_russia_data/gross_regional_product_1996_2020.xls', 
                    header=2, skipfooter=1, usecols='A, Q:Z')

gdpp.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']

gdpp = fix_name_reg(gdpp, region_names, names_to_drop)
gdpp.columns = pd.to_datetime(gdpp.columns).year

display('Валовой региональный продукт на душу населения, руб.', gdpp.head(3))
display('Колличество регионов - %.f' % gdpp.shape[0])

'Валовой региональный продукт на душу населения, руб.'

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,
Алтайский край,137677.2,153556.7,173763.5,186798.6,204933.1,224525.8,231268.4,247599.3,271319.7,291156.9
Амурская область,273175.8,280023.9,258817.0,286282.6,343385.7,370192.4,373935.1,419905.2,521060.1,571362.1
Архангельская область без Ненецкого АО,232540.7,270662.9,283264.5,310817.4,352837.9,400764.6,441961.6,493205.1,509917.0,514200.4


'Колличество регионов - 85'

#### 2.6. Бюджеты регионов за 2020г.

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru)

***income*** - средства, получаемые из налогов, акцизов, использования имущества;
***subsidies*** - дотации из бюджетов других уровней (зачастую федерального).

In [8]:
df = pd.read_excel('social_russia_data/regional_budget_incomes_2020.xlsx', 
                        header=2, usecols='A, C, E, G, I, K, M, O')
df['income'] = df[df.columns[-7:-1]].sum(axis=1) / 1e6 
df['subsidies'] = df[df.columns[-2]] / 1e6 
df.drop(columns=df.columns[-9:-2], inplace=True)
df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
income = fix_name_reg(df, region_names)

display('Доходы бюджета; млн руб.б', income.head(3))
display('Колличество регионов - %.f' % income.shape[0])

'Доходы бюджета; млн руб.б'

,income,subsidies
region,,
Алтайский край,56637.879730,74306.235370
Амурская область,51571.555319,34011.885547
Архангельская область без Ненецкого АО,52963.993010,42447.952872


'Колличество регионов - 85'

#### 2.7. Расходы на социальную политику от общих расходов бюджета региона за период 2015-2020гг., % в год

In [9]:
def welfared(file, sheet):    
    # Считывание финансовых данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    names_to_drop = []
    df = fix_name_reg(df, region_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

file = './social_russia_data/welfare_expense_share_2015_2020.xlsx'

welfare = welfared(file, 0)

display('Расходы на социальную политику от общих расходов бюджета региона, %.', welfare.head(3))
display('Колличество регионов - %.f' % welfare.shape[0])

'Расходы на социальную политику от общих расходов бюджета региона, %.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,18.8,20.4,30.9,29.8,28.3,29.7
Амурская область,19.2,28.9,26.3,24.1,22.7,21.7
Архангельская область без Ненецкого АО,15.4,16.3,24.8,24.2,22.4,21.4


'Колличество регионов - 85'

#### 2.8. Средний объем отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности за период 2017-2020гг, млн.руб. ####

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru/enterprise_industrial):

 * Данные регионов приведены по полному кругу организаций по фактическим видам экономической деятельности;

 * Данные некоторых регионов уточнены с учетом итогов сплошного наблюдения за деятельностью субъектов малого и среднего предпринимательства за 2020 год;
 
 * Данные некоторых регионов не публикуются в целях обеспечения конфиденциальности первичных статистических данных, полученных от  организаций, в соответствии с Федеральным законом от 29.11.07 № 282-ФЗ "Об официальном статистическом учете и  системе государственной статистики в Российской Федерации" (п.5, ст.4; ч.1, ст.9).

 Пропущенные данные были заменены нулевыми значениями при их отсутствии за весь период.

In [10]:
reg_prod = pd.read_csv(
    'social_russia_data/regional_production_2017_2020.csv')
names_to_drop = [
    '        Архангельская область', 
    '        Тюменская область']
reg_prod = fix_name_reg(reg_prod, region_names, names_to_drop) 
reg_prod.columns.values[-4:] = pd.to_datetime(reg_prod.columns[-4:]).year

# создадим признак  'Объем добычи полезных ископаемых'
mining = reg_prod[reg_prod['production_field'] == 'ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ'].loc[:,2017:2020]
# создадим признак 'Объем обрабатывающих производств'
manufact = reg_prod[reg_prod['production_field'] == 'ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА'].loc[:,2017:2020]
# создадим признак 'Объем промышленного производства'
indust = reg_prod[reg_prod['production_field'] == 'Промышленное производство (промышленность)'].loc[:,2017:2020]
# создадим признак 'Объем водоснабжения, организации сбора и утилизации отходов'
water_supply = reg_prod[reg_prod['production_field'] == 'ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ'].loc[:,2017:2020]
#создадим признак 'Объем обеспечения электроэнергией, газом и паром'
provision = reg_prod[reg_prod['production_field'] == 'ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА'].loc[:,2017:2020]

reg_prod_total = (reg_prod
                  .groupby('region')[[2017, 2018, 2019, 2020]].sum()) 

reg_prod_total.columns = pd.to_datetime(
    reg_prod_total.columns, format='%Y').year

#Переведем значения в руб
mining=mining*1e6
manufact=manufact*1e6
indust=indust*1e6
water_supply=water_supply*1e6
provision=provision*1e6
reg_prod_total=reg_prod_total*1e6

display('Общий объем регионалбного производства, руб.', reg_prod_total.head(3))

display('Колличество регионов - %.f' % reg_prod_total.shape[0]) 


'Общий объем регионалбного производства, руб.'

,2017,2018,2019,2020
region,,,,
Алтайский край,7.359764e+14,7.711927e+14,8.383179e+14,8.524434e+14
Амурская область,2.688441e+14,2.687608e+14,3.400267e+14,4.045765e+14
Архангельская область без Ненецкого АО,7.090098e+14,6.064711e+14,5.335849e+14,6.641070e+14


'Колличество регионов - 85'

#### 2.9. Средняя розничная торговля на душу населения за период 2013-2021гг., млн.руб

In [11]:
df = pd.read_excel(
    'social_russia_data/retail_turnover_per_capita_2000_2021.xls', 
    header=2, usecols='A, P:X')
df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
df.drop(0, inplace=True)
names_to_drop = [
        '        Архангельская область',
        '            Архангельская область (без АО)',
        '        Тюменская область',
        '            Тюменская область (без АО)'
]
retail = fix_name_reg(df, region_names, names_to_drop)
retail.columns = pd.to_datetime(retail.columns).year

display('Объем розничной торговли на душу населения за год, млн.руб', retail.head(3))
display('Колличество регионов - %.f' % retail.shape[0])


'Объем розничной торговли на душу населения за год, млн.руб'

,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Алтайский край,118096,128376,134925,137844,143873,150444,159514,153605,174321
Амурская область,145301,163781,182491,191523,202038,214688,231113,245233,276635
Архангельская область без Ненецкого АО,154177,176420,194345,202977,217332,229922,240155,250033,280050


'Колличество регионов - 85'

#### 2.11. Доля людей, живущих за чертой бедности от общего числа населения региона за период 1992-2020гг., %

In [12]:
df = pd.read_csv('social_russia_data/poverty_percent_by_regions_1992_2020.csv')

df = pd.pivot_table(
        data=df,
        values='poverty_percent',
        columns='year',
        index='region').reset_index()

names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']

poverty_percent = fix_name_reg(df, region_names, names_to_drop)

poverty_percent.columns = pd.to_datetime(poverty_percent.columns, format='%Y').year

display('Процент населения региона, живущих за чертой бедности', poverty_percent.head(3))
display('Колличество регионов - %.f' % poverty_percent.shape[0])

'Процент населения региона, живущих за чертой бедности'

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,NaN,NaN,NaN,33.7,46.8,45.7,52.9,53.8,53.9,47.3,...,22.6,20.6,17.6,17.1,18.0,17.8,17.5,17.4,17.6,17.5
Амурская область,NaN,NaN,NaN,36.1,28.2,26.3,31.2,38.0,47.7,45.3,...,20.4,16.0,16.2,14.8,15.2,17.0,16.7,15.6,15.7,15.2
Архангельская область без Ненецкого АО,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3


'Колличество регионов - 85'

#### 2.12. Распределение живущих за чертой бедности по социально-демографическим группам за период 2017-2020гг, %

In [13]:
def socdem(file):    
    df = pd.read_excel(file, header=2)    
    df.rename({
        'Unnamed: 0': 'region',
        'Все население': 'all',
        'Дети в возрасте до 16 лет': 'children',
        'Население трудоспособного возраста': 'employable',
        'Население старше трудоспособного возраста': 'pensioner'        
        }, axis=1, inplace=True)    
    names_to_drop = [
        '        Архангельская область',
        '        Тюменская область'
    ]    
    df = fix_name_reg(df, region_names, names_to_drop)    
    return df

ps_2017 = socdem('social_russia_data/poverty_socdem_2017.xls')
ps_2018 = socdem('social_russia_data/poverty_socdem_2018.xls')
ps_2019 = socdem('social_russia_data/poverty_socdem_2019.xls')
ps_2020 = socdem('social_russia_data/poverty_socdem_2020.xls')

In [14]:
# Создадим признак 'Доля малоимущих детей'
ps_17_child = ps_2017.loc[:, ['children']]
ps_18_child = ps_2018.loc[:, ['children']]
ps_19_child = ps_2019.loc[:, ['children']]
ps_20_child = ps_2020.loc[:, ['children']]

ps_17_child.rename(columns={'children':'2017'},inplace=True)
ps_18_child.rename(columns={'children':'2018'},inplace=True)
ps_19_child.rename(columns={'children':'2019'},inplace=True)
ps_20_child.rename(columns={'children':'2020'},inplace=True)

ps_child = ps_17_child.merge(ps_18_child,on=['region'],how='left')
ps_child = ps_child.merge(ps_19_child,on=['region'],how='left')
ps_child = ps_child.merge(ps_20_child,on=['region'],how='left')

ps_child.columns = pd.to_datetime(ps_child.columns, format='%Y').year

display('Доля детей в возрасте до 16 лет живущих за чертой бедности, %', ps_child.head(3))
display('Колличество регионов - %.f' % ps_child.shape[0])

'Доля детей в возрасте до 16 лет живущих за чертой бедности, %'

,2017,2018,2019,2020
region,,,,
Алтайский край,38.0,42.1,38.7,31.4
Амурская область,39.9,40.6,33.9,38.4
Архангельская область без Ненецкого АО,37.4,38.8,31.3,28.5


'Колличество регионов - 85'

In [15]:
# создадим признак 'Доля малоимущих граждан трудоспособного возраста'
ps_17_empl = ps_2017.loc[:, ['employable']]
ps_18_empl = ps_2018.loc[:, ['employable']]
ps_19_empl = ps_2019.loc[:, ['employable']]
ps_20_empl = ps_2020.loc[:, ['employable']]

ps_17_empl.rename(columns={'employable':'2017'},inplace=True)
ps_18_empl.rename(columns={'employable':'2018'},inplace=True)
ps_19_empl.rename(columns={'employable':'2019'},inplace=True)
ps_20_empl.rename(columns={'employable':'2020'},inplace=True)

ps_empl = ps_17_empl.merge(ps_18_empl,on=['region'],how='left')
ps_empl = ps_empl.merge(ps_19_empl,on=['region'],how='left')
ps_empl = ps_empl.merge(ps_20_empl,on=['region'],how='left')

ps_empl.columns = pd.to_datetime(ps_empl.columns, format='%Y').year

display('Доля населения трудоспособного возраста живущих за чертой бедности, %', ps_empl.head(3))
display('Колличество регионов - %.f' % ps_empl.shape[0])

'Доля населения трудоспособного возраста живущих за чертой бедности, %'

,2017,2018,2019,2020
region,,,,
Алтайский край,55.6,52.8,57.4,59.4
Амурская область,54.1,54.8,58.9,56.9
Архангельская область без Ненецкого АО,55.5,54.6,59.8,64.3


'Колличество регионов - 85'

In [16]:
# Создадим признак 'Доля малоимущих граждан  пенсионного возраста'
ps_17_pens = ps_2017.loc[:, ['pensioner']]
ps_18_pens = ps_2018.loc[:, ['pensioner']]
ps_19_pens = ps_2019.loc[:, ['pensioner']]
ps_20_pens = ps_2020.loc[:, ['pensioner']]

ps_17_pens.rename(columns={'pensioner':'2017'},inplace=True)
ps_18_pens.rename(columns={'pensioner':'2018'},inplace=True)
ps_19_pens.rename(columns={'pensioner':'2019'},inplace=True)
ps_20_pens.rename(columns={'pensioner':'2020'},inplace=True)

ps_pens = ps_17_pens.merge(ps_18_pens,on=['region'],how='left')
ps_pens = ps_pens.merge(ps_19_pens,on=['region'],how='left')
ps_pens = ps_pens.merge(ps_20_pens,on=['region'],how='left')

ps_pens.columns = pd.to_datetime(ps_pens.columns, format='%Y').year

display('Доля населения пенсионного возраста живущих за чертой бедности, %', ps_pens.head(3))
display('Колличество регионов - %.f' % ps_pens.shape[0])

'Доля населения пенсионного возраста живущих за чертой бедности, %'

,2017,2018,2019,2020
region,,,,
Алтайский край,6.4,5.1,3.9,9.2
Амурская область,6.0,4.6,7.2,4.7
Архангельская область без Ненецкого АО,7.1,6.6,8.9,7.2


'Колличество регионов - 85'

#### 2.13. Социальные признаки детей проживающих в регионе за период 2020-2021г.

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru)

In [17]:
def get_child(file):    
    df = pd.read_excel(file, header=1, usecols='B:E')    
    df.rename({
        'Регион': 'region',
        'Всего детей в регионе, человек': 'childrens',
        'Нуждаются в устройстве в семьи, человек': 'orphanage',
        'Устройство детей в семьи, %': 'adopted %'        
        }, axis=1, inplace=True)    
    names_to_drop = [
        '        Архангельская область',
        '        Тюменская область'
    ]    
    df = fix_name_reg(df, region_names, names_to_drop)    
    return df

ch_2020 = get_child('social_russia_data/report_orphanhood_2020.xlsx')
ch_2021 = get_child('social_russia_data/report_orphanhood_2021.xlsx')

In [18]:
# создадим признак 'Колличество детей в регионе'
ch_2020_childrens = ch_2020.loc[:, ['childrens']]
ch_2021_childrens = ch_2021.loc[:, ['childrens']]

ch_2020_childrens.rename(columns={'childrens':'2020'},inplace=True)
ch_2021_childrens.rename(columns={'childrens':'2021'},inplace=True)

childrens = ch_2020_childrens.merge(ch_2021_childrens,on=['region'],how='left')

display('Колличество детей в регионе; чел.', childrens.head(3))
display('Колличество регионов - %.f' % childrens.shape[0])

'Колличество детей в регионе; чел.'

,2020,2021
region,,
Алтайский край,490149,485005
Амурская область,178702,177105
Архангельская область без Ненецкого АО,226228,222724


'Колличество регионов - 85'

In [19]:
# создадим признак 'Колличество детей нуждаются в устройстве в семьи'
ch_2020_childrens = ch_2020.loc[:, ['orphanage']]
ch_2021_childrens = ch_2021.loc[:, ['orphanage']]

ch_2020_childrens.rename(columns={'orphanage':'2020'},inplace=True)
ch_2021_childrens.rename(columns={'orphanage':'2021'},inplace=True)

child_orphanage = ch_2020_childrens.merge(ch_2021_childrens,on=['region'],how='left')

display('Колличество детей нуждаются в устройстве в семьи в регионе; чел.', child_orphanage.head(3))
display('Колличество регионов - %.f' % child_orphanage.shape[0])

'Колличество детей нуждаются в устройстве в семьи в регионе; чел.'

,2020,2021
region,,
Алтайский край,983,1034
Амурская область,676,670
Архангельская область без Ненецкого АО,785,770


'Колличество регионов - 85'

In [20]:
# создадим признак 'Доля детей устроенных в семьи от нуждающихся'
ch_2020_childrens = ch_2020.loc[:, ['adopted %']]
ch_2021_childrens = ch_2021.loc[:, ['adopted %']]

ch_2020_childrens.rename(columns={'adopted %':'2020'},inplace=True)
ch_2021_childrens.rename(columns={'adopted %':'2021'},inplace=True)

child_adopted = ch_2020_childrens.merge(ch_2021_childrens,on=['region'],how='left')

display('Колличество детей нуждаются в устройстве в семьи в регионе; чел.', child_adopted.head(3))
display('Колличество регионов - %.f' % child_adopted.shape[0])

'Колличество детей нуждаются в устройстве в семьи в регионе; чел.'

,2020,2021
region,,
Алтайский край,59,57
Амурская область,65,62
Архангельская область без Ненецкого АО,60,64


'Колличество регионов - 85'

#### 2.14. Социальные признаки пенсионеров проживающих в регионе за период 2015-2021г.

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru)

In [21]:
pens_df = pd.read_csv(
    'social_russia_data/problem_ageing_all.csv', decimal=',')
cols = [
    'Название показателя', 'Ед.измерения', 'Название региона', 'Сегмент',
    '2015', '2016', '2017', '2018', '2019', '2020', '2021'
]
pens_df = pens_df[cols]
pens_df.replace('н/д', np.NaN, inplace=True)
pens_df.replace('#Н/Д', np.NaN, inplace=True)
pens_df.columns.values[-7:] = pd.to_datetime(pens_df.columns[-7:]).year
pens_df.rename({'Название региона': 'region'}, axis=1, inplace=True)

In [22]:
# создадим признак 'Размер пенсии'
mask = pens_df['Название показателя'] == 'Средний размер пенсии'
pension_df = pens_df[mask].copy()
pension_df.drop(columns=['Название показателя', 'Ед.измерения', 'Сегмент', 2021],
                inplace=True) 
for year in range(2015, 2021):
    pension_df[year] = (pension_df[year]
                     .apply(lambda x: float(str(x).replace(',','.'))))

# сложим страховую и наскопительную часть пенсии
pension_df = pension_df.groupby('region').sum().reset_index()  
pension = fix_name_reg(pension_df, region_names)

display('Размер пенсии; руб.', pension.head(3))
display('Колличество регионов - %.f' % pension.shape[0])

'Размер пенсии; руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,12198.70,12523.82,13147.23,14376.00,15243.87,16128.19
Амурская область,13698.68,14092.91,14780.44,16065.37,17005.85,17998.70
Архангельская область без Ненецкого АО,0.00,0.00,0.00,0.00,0.00,21753.88


'Колличество регионов - 85'

In [23]:
# # создадим признак 'Колличество пенсионеров региона'
mask = ((pens_df['Название показателя'] == 'Люди в возрасте старше трудоспособного')
        & (pens_df['Ед.измерения'] == 'человек') & (pens_df['Сегмент'] == 'всего'))
pensioners = pens_df[mask].copy()
pensioners.drop(columns=['Название показателя', 'Ед.измерения', 'Сегмент'],
                     inplace=True) 
for year in range(2015, 2022):
    pensioners[year] = (pensioners[year]
                     .apply(lambda x: float(str(x).replace(',','.'))))
pensioners = fix_name_reg(pensioners, region_names)

display('Колличество пенсионеров региона; %', pensioners.head(3))
display('Колличество регионов - %.f' % pensioners.shape[0])

'Колличество пенсионеров региона; %'

,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,
Алтайский край,604333.0,618276.0,629725.0,639803.0,647643.0,621984.0,622583.0
Амурская область,175375.0,178772.0,181429.0,183228.0,185016.0,176633.0,176179.0
Архангельская область без Ненецкого АО,283394.0,288882.0,293477.0,297292.0,300715.0,288908.0,290570.0


'Колличество регионов - 85'

In [24]:
# создадим признак 'Доля пенсионеров от населения региона'
mask = ((pens_df['Название показателя'] == 'Люди в возрасте старше трудоспособного')
        & (pens_df['Ед.измерения'] == '%') & (pens_df['Сегмент'] == 'всего'))
pension_percent = pens_df[mask].copy()
pension_percent.drop(columns=['Название показателя', 'Ед.измерения', 'Сегмент'],
                     inplace=True) 
for year in range(2015, 2022):
    pension_percent[year] = (pension_percent[year]
                     .apply(lambda x: float(str(x).replace(',','.'))))
pensioners_percent = fix_name_reg(pension_percent, region_names)

display('Доля пенсионеров от населения региона; %', pensioners_percent.head(3))
display('Колличество регионов - %.f' % pensioners_percent.shape[0])

'Доля пенсионеров от населения региона; %'

,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,
Алтайский край,25.3,26.0,26.6,27.2,27.8,26.8,27.1
Амурская область,21.7,22.2,22.6,22.9,23.3,22.4,22.5
Архангельская область без Ненецкого АО,33.0,25.6,26.2,26.8,27.3,26.4,26.8


'Колличество регионов - 85'

#### 2.15. Прожиточный минимум.

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru) за IV квартал 2020г.

In [25]:
df = pd.read_excel('social_russia_data/living_wage_2020.xls',
                            header=4, usecols='A, I')
df.rename({
    'Unnamed: 0': 'region',
    'Трудоспособное население.3': 'living wage',
    }, axis=1, inplace=True)
names_to_drop = ['        Архангельская область',
                 '        Тюменская область']
living_wage = fix_name_reg(df, region_names, names_to_drop)
living_wage.rename(columns={'living wage':'2020'},inplace=True)
living_wage.columns = pd.to_datetime(living_wage.columns, format='%Y').year

display('Прожиточный минимум региона; руб.', living_wage.head(3))
display('Колличество регионов - %.f' % pensioners_percent.shape[0])

'Прожиточный минимум региона; руб.'

,2020
region,
Алтайский край,10904.0
Амурская область,14331.0
Архангельская область без Ненецкого АО,14507.0


'Колличество регионов - 85'

#### 2.16. Уровень безработицы в регионах РФ за преиод 2000-2021гг., % ####

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru/labour_force)

In [26]:
unemployment_rate = pd.read_excel('social_russia_data/trud_15-72.xlsx')

names_to_drop = ['Архангельская область', 'Тюменская область']

unemployment_rate = fix_name_reg(unemployment_rate, region_names, names_to_drop)

display('Уровень безработицы, %', unemployment_rate.head(3))
display('Колличество регионов - %.f' % unemployment_rate.shape[0])

'Уровень безработицы, %'

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,11.7,10,8.3,12.3,9.1,8.8,8.9,6.4,8.3,12.3,...,6.2,8.3,7.2,8.0,8.6,6.9,6.1,5.8,5.9,5.5
Амурская область,13.7,12.3,10.7,9.8,11.7,9.9,8.2,6.1,5.4,8.3,...,5.3,6.1,5.6,5.8,5.9,5.9,5.6,5.4,6.0,5.2
Архангельская область без Ненецкого АО,12.4,8.6,8.1,10,7.1,5.3,5.9,5.3,6.8,7.1,...,5.3,6.0,7.3,6.8,7.1,6.4,6.3,6.2,7.4,6.6


'Колличество регионов - 85'

#### 2.17. Рождаемость за период 2006-2022гг. ####

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru/labour_force)

In [27]:
df = pd.read_csv('social_russia_data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')

df.drop(columns='Unnamed: 198',inplace=True)

df.rename({'Region': 'region'}, axis=1, inplace=True)

names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]

newborn = fix_name_reg(df, region_names, names_to_drop=names_to_drop)

newborn.loc['Архангельская область без Ненецкого АО'] \
    = (newborn.loc['Архангельская область без Ненецкого АО']
      - newborn.loc['Ненецкий автономный округ'])
    
newborn.loc['Тюменская область без округов'] \
    = (newborn.loc['Тюменская область без округов']
       - newborn.loc['Ханты-Мансийский автономный округ - Югра']
       - newborn.loc['Ямало-Ненецкий автономный округ'])
    
# группируем данные по годам 
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
    
newborn.drop(columns=2022, inplace=True)  # удаляем 2022г., т.к. данные не за все месяца

newborn = newborn.round(0).astype(int)

newborn.columns = pd.to_datetime(newborn.columns, format='%Y').year

display('Колличество родившихся; чел', newborn.head(3))
display('Колличество регионов - %.f' % newborn.shape[0])

'Колличество родившихся; чел'

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,
Алтайский край,25513,28209,30711,31038,30863,30739,32695,32461,31429,30065,28867,25525,23491,21117,19976,18992
Амурская область,10405,10935,11233,11384,11473,11188,11733,11483,11159,10781,10377,9430,8889,7919,7853,7343
Архангельская область без Ненецкого АО,13396,14438,14577,14776,14806,14253,14730,14584,14331,13908,13313,11714,10671,9662,8906,8487


'Колличество регионов - 85'

#### 2.18. Детская смертность за период 1990-2021гг

In [28]:
def mortality(file):
    
    # Считывание данных с excel-листа
    df = pd.read_excel(file, header=2, skipfooter=13, usecols='A, D:AI')
    
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    names_to_drop = [
        '            Архангельская область (кроме Ненецкого автономного округа)',
        '            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    ]
    
    df = fix_name_reg(df, region_names, names_to_drop)
    df.loc['Архангельская область без Ненецкого АО'] \
        = (df.loc['Архангельская область без Ненецкого АО']
           - df.loc['Ненецкий автономный округ'])
        
    df.loc['Тюменская область без округов'] \
        = (df.loc['Тюменская область без округов']
           - df.loc['Ханты-Мансийский автономный округ - Югра']
           - df.loc['Ямало-Ненецкий автономный округ'])
        
    df.columns = pd.to_datetime(df.columns, format='%Y').year
    
    return df

# смертность детей в сельской местности на первом году жизни за год
child_mortality_rural = mortality('social_russia_data/child_mortality_rural_1990_2021.xls')
# смертность детей в городской местности на первом году жизни за год
child_mortality_urban = mortality('social_russia_data/child_mortality_urban_1990_2021.xls')

# смертность детей на первом году жизни за год
child_mortality = child_mortality_rural.add(child_mortality_urban)

display('Смертность детей на первом году жизни за год ', child_mortality.head(3))
display('Колличество регионов - %.f' % child_mortality.shape[0])

'Смертность детей на первом году жизни за год '

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,522.0,567.0,529.0,560.0,497.0,490.0,405.0,375.0,340.0,391.0,...,330.0,290.0,275.0,222.0,211.0,185.0,176.0,114.0,97.0,88.0
Амурская область,343.0,293.0,280.0,244.0,242.0,250.0,245.0,268.0,246.0,270.0,...,161.0,119.0,101.0,83.0,53.0,48.0,46.0,43.0,42.0,36.0
Архангельская область без Ненецкого АО,288.0,274.0,245.0,237.0,224.0,206.0,194.0,157.0,167.0,168.0,...,108.0,113.0,97.0,84.0,79.0,72.0,53.0,58.0,29.0,32.0


'Колличество регионов - 85'

#### 2.19. Среднее число зарегистрированных заболеваний у пациентов с диагнозом, установленном впервые в жизни (без данных Минздрава России), на конец года за период 2011-2021гг., чел. ####

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru/folder/13721)

In [29]:
def morbidityd(file):
    
    # Считывание финансовых данных с excel-листа"
    df = pd.read_excel(file)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = fix_name_reg(df, region_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

morbidity = morbidityd('social_russia_data/number of diseases.xls')

display(' Число зарегистрированных заболеваний у пациентов с диагнозом, установленном впервые в жизни ', morbidity.head(3))
display('Колличество регионов - %.f' % morbidity.shape[0])

' Число зарегистрированных заболеваний у пациентов с диагнозом, установленном впервые в жизни '

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,
Алтайский край,204764,161291.0,189203.0,281414,257099,287363,264641,307326,332416,335066,415826
Амурская область,185194,164643.0,186843.0,167295,156200,148486,167661,162356,189366,164545,224767
Архангельская область без Ненецкого АО,319568,450922.0,456385.0,472621,491481,477407,476199,459615,354760,565545,686681


'Колличество регионов - 85'

#### 2.20. Среднее число респираторных заболеваний (на 1тыс человек) за период 2016-2020гг. ####

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru/folder/13721)

In [30]:
def respiratory_diseases(file):
    
    # Считывание данных с excel-листа"
    df = pd.read_excel(file)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = fix_name_reg(df, region_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

respiratory = respiratory_diseases('social_russia_data/respiratory_diseases.xlsx')

display(' Среднее число респираторных заболеваний', respiratory.head(3))
display('Колличество регионов - %.f' % respiratory.shape[0])

' Среднее число респираторных заболеваний'

,2016,2017,2018,2019,2020
region,,,,,
Алтайский край,35.625,38.479,48.489,42.961,57.302
Амурская область,30.066,29.156,28.982,26.776,30.788
Архангельская область без Ненецкого АО,77.415,76.284,72.306,54.373,88.038


'Колличество регионов - 85'

#### 2.21. Среднее количество случаев онкологических заболеваний  (на 1тыс человек) за период 2016-2020гг. ####

Данные взяты с ресурса ['https://rosstat.gov.ru'](https://rosstat.gov.ru/folder/13721)

In [31]:
def neoplasms_diseases(file, sheet):
    
    # Считывание данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    names_to_drop = []
    df = fix_name_reg(df, region_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

file = './social_russia_data/neoplasms.xlsx'

neoplasms = neoplasms_diseases(file, 0)

display(' Среднее число онкологических заболеваний заболеваний', neoplasms.head(3))
display('Колличество регионов - %.f' % neoplasms.shape[0])

' Среднее число онкологических заболеваний заболеваний'

,2016,2017,2018,2019,2020
region,,,,,
Алтайский край,17.768,15.096,13.992,15.855,10.293
Амурская область,3.940,4.199,3.652,2.446,3.568
Архангельская область без Ненецкого АО,18.928,15.279,14.851,9.918,16.234


'Колличество регионов - 85'

#### 2.22. Инвалидность среди трудоспособного возраста

In [32]:
df = pd.read_csv(
    'social_russia_data/disabled_total_by_age_2017_2022.csv')
df = fix_name_reg(df, region_names)

df['date'] = pd.to_datetime(df['date'])
df['dayofyear'] = df['date'].dt.dayofyear
df = df[df['dayofyear'] == 1]
df['young'] = df[['18_30', '31_40', '41_50', '51_60']].sum(axis=1)
disabled_18_60 = pd.pivot_table(
    data=df, values='young', index='region', columns='date')
disabled_18_60.columns = (disabled_18_60.columns).year

display('Колличество инвалидов трудоспособного возраста (18-60лет) в регионе; чел.', disabled_18_60.head(3))
display('Колличество регионов - %.f' % disabled_18_60.shape[0])

'Колличество инвалидов трудоспособного возраста (18-60лет) в регионе; чел.'

date,2017,2018,2019,2020,2021,2022
region,,,,,,
Алтайский край,66770.0,64605.0,62840.0,61378.0,59810.0,59006.0
Амурская область,27892.0,26720.0,25976.0,25069.0,24149.0,23356.0
Архангельская область без Ненецкого АО,29228.0,28262.0,27615.0,26984.0,26498.0,26009.0


'Колличество регионов - 85'

### <center> 3. Создание обобщенной базы данных. 

#### 3.1. Сборка excel-книги листы которой заполнены подготовленными выше данными признаков.  

In [33]:

with ExcelWriter('./data/soc_pasport.xlsx') as writer:
    population.to_excel(writer, sheet_name='population')
    area.to_excel(writer, sheet_name='area')
    living_area.to_excel(writer, sheet_name='living_area')
    per_capita.to_excel(writer, sheet_name='per_capita')
    real_incomes.to_excel(writer, sheet_name='real_incomes')
    formal_wage.to_excel(writer, sheet_name='formal_wage')
    real_pay.to_excel(writer, sheet_name='real_pay')
    gdpp.to_excel(writer, sheet_name='gdpp')
    income.to_excel(writer, sheet_name='income')
    welfare.to_excel(writer, sheet_name='welfare') 
    mining.to_excel(writer, sheet_name='mining')  
    manufact.to_excel(writer, sheet_name='manufact')
    indust.to_excel(writer, sheet_name='indust')
    water_supply.to_excel(writer, sheet_name='water_supply')
    provision.to_excel(writer, sheet_name='provision')
    reg_prod_total.to_excel(writer, sheet_name='reg_prod_total')
    retail.to_excel(writer, sheet_name='retail')
    poverty_percent.to_excel(writer, sheet_name='poverty_percent')
    ps_child.to_excel(writer, sheet_name='ps_child')
    ps_empl.to_excel(writer, sheet_name='ps_empl')
    ps_pens.to_excel(writer, sheet_name='ps_pens') 
    childrens.to_excel(writer, sheet_name='childrens') 
    child_orphanage.to_excel(writer, sheet_name='child_orphanage') 
    child_adopted.to_excel(writer, sheet_name='child_adopted') 
    pension.to_excel(writer, sheet_name='pension') 
    pensioners.to_excel(writer, sheet_name='pensioners') 
    pensioners_percent.to_excel(writer, sheet_name='pensioners_percent') 
    living_wage.to_excel(writer, sheet_name='living_wage')
    unemployment_rate.to_excel(writer, sheet_name='unemployment_rate')
    newborn.to_excel(writer, sheet_name='newborn')
    child_mortality_rural.to_excel(writer, sheet_name='child_mortality_rural')
    child_mortality_urban.to_excel(writer, sheet_name='child_mortality_urban')
    child_mortality.to_excel(writer, sheet_name='child_mortality')
    morbidity.to_excel(writer, sheet_name='morbidity')
    respiratory.to_excel(writer, sheet_name='respiratory')
    neoplasms.to_excel(writer, sheet_name='neoplasms')
    disabled_18_60.to_excel(writer, sheet_name='disabled_18_60')    
    

#### 3.2. Создание датафрейма данных. ####

Представленные данные (кроме признака area) имеют вид временных рядов. С целью воспроизводимости моделирования возьмем данные по общему срезу - за 2020 год. Признак area является статичным и с течением времени не меняется.  

In [34]:
# Сформируем датасет из созданной excel-книги по условию "за 2020 год" с помощью созданного инструмента tools.add_col и сохраним его в формате csv:
file = './data/soc_pasport.xlsx'

df = pd.concat([add_col(file, 'population', 'W'),
                        add_col(file, 'area', 'B'),
                        add_col(file, 'living_area', 'B'),
                        add_col(file, 'per_capita', 'G'),
                        add_col(file, 'real_incomes', 'G'),
                        add_col(file, 'formal_wage', 'G'),
                        add_col(file, 'real_pay', 'G'),
                        add_col(file, 'gdpp', 'K'),
                        add_col(file, 'income'),
                        add_col(file, 'welfare', 'G'),
                        add_col(file, 'mining', 'E'),
                        add_col(file, 'manufact', 'E'),
                        add_col(file, 'indust', 'E'),
                        add_col(file, 'water_supply', 'E'),
                        add_col(file, 'provision', 'E'),
                        add_col(file, 'reg_prod_total', 'E'),
                        add_col(file, 'retail', 'I'),
                        add_col(file, 'poverty_percent', 'AD'),
                        add_col(file, 'ps_child', 'E'),
                        add_col(file, 'ps_empl', 'E'),
                        add_col(file, 'ps_pens', 'E'),
                        add_col(file, 'childrens', 'B'),
                        add_col(file, 'child_orphanage', 'B'),
                        add_col(file, 'child_adopted', 'B'),
                        add_col(file, 'pension', 'G'),
                        add_col(file, 'pensioners', 'G'),
                        add_col(file, 'pensioners_percent', 'G'),
                        add_col(file, 'living_wage', 'B'),
                        add_col(file, 'unemployment_rate', 'V'),
                        add_col(file, 'newborn', 'P'),
                        add_col(file, 'child_mortality_rural', 'AF'),
                        add_col(file, 'child_mortality_urban', 'AF'),
                        add_col(file, 'child_mortality', 'AF'),
                        add_col(file, 'morbidity', 'K'),
                        add_col(file, 'respiratory', 'F'),
                        add_col(file, 'neoplasms', 'F'),
                        add_col(file, 'disabled_18_60', 'E')                       
    ],
    axis=1).fillna(0).set_index('region')

#### 3.3. Дополнение данных удельными значениями некоторых признаков: ####

In [35]:
# добавим плотность населения региона в 2020г.
df['density'] = df['population'] / df['area']

In [36]:
# добавим удельный объем регионального производства на душу населения
df['spec_mining'] = df['mining'] / df['population']
df['spec_manufact'] = df['manufact'] / df['population']
df['spec_indust'] = df['indust'] / df['population']
df['spec_water_supply'] = df['water_supply'] / df['population']
df['spec_provision'] = df['provision'] / df['population']
df['spec_reg_prod'] = df['reg_prod_total'] / df['population']

#### 3.4. Сохранение полученного набора данных

In [37]:
df.to_csv('./data/sp_df_2020.csv')
df.head(3)

,population,area,living_area,per_capita,real_incomes,formal_wage,real_pay,gdpp,income,subsidies,...,respiratory,neoplasms,disabled_18_60,density,spec_mining,spec_manufact,spec_indust,spec_water_supply,spec_provision,spec_reg_prod
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2317153,167996,18.0,23864,95.5,30072,103.8,291156.9,56637.879730,74306.235370,...,57.302,10.293,61378,13.792906,3.947148e+06,1.561745e+08,1.839420e+08,4.511748e+06,1.930862e+07,3.678839e+08
Амурская область,790044,361908,17.3,35499,100.3,52430,105.2,571362.1,51571.555319,34011.885547,...,30.788,3.568,25069,2.182997,1.477420e+08,4.948314e+07,2.560468e+08,4.393894e+06,5.442775e+07,5.120936e+08
Архангельская область без Ненецкого АО,1092424,413103,16.9,34852,98.4,52779,102.9,514200.4,52963.993010,42447.952872,...,88.038,16.234,26984,2.644435,2.810097e+07,2.372191e+08,3.039603e+08,5.356946e+06,3.328323e+07,6.079205e+08


**Описание набора данных:**

 * **population** - численность населения региона (чел.);

 * **area** - площадь региона (кв.км);

 * **living area** - средняя жилая площадь на одного человека;

 * **per capita** - среднедушевые денежные доходы (в месяц), (руб.);

 * **real_incomes** - реальные денежные доходы, в процентах к предыдущему году (покупательная способность: с учетом роста цен), (руб.);

 * **formal wage** - среднемесячная номинальная начисленная заработная плата (руб.);

 * **real_pay** - реальная начисленная заработная плата, в процентах к предыдущему году (покупательная способность: с учетом роста цен) (руб.);

 * **gdpp** - валовой региональный продукт на душу населения (руб.);

 * **income** - доходы региона, получаемые из налогов, акцизов, использования имущества(руб.);

 * **subsidies** - дотации из бюджетов других уровней (зачастую федерального) (руб.); 
 
 * **welfare** - расходы на социальную политику от общих расходов бюджета региона (%);

 * **mining** - объем добычи полезных ископаемых (млн.руб.);

 * **manufact** - объем обрабатывающих производств (млн.руб.);

 * **indust** - объем промышленного производства (млн.руб.);

 * **water_supply** - объем водоснабжения, организации сбора и утилизации отходов (млн.руб.);
 
 * **provision** - объем обеспечения электроэнергией, газом и паром (млн.руб.);

 * **reg_prod_total** - общий объем регионального производства (млн.руб.);

 * **retail** - объем розничной торговли на душу населения за год (млн.руб);

 * **poverty_percent** - доля населения региона, живущих за чертой бедности (%);

 * **ps_child** - доля детей в возрасте до 16 лет живущих за чертой бедности (%);

 * **ps_empl** - доля людей работоспособного возраста (16-60 лет) живущих за чертой бедности (%);

 * **ps_pens** - доля людей пенсионного возраста (более 60 лет) живущих за чертой бедности (%);

 * **childrens** - колличество детей в регионе (чел.);

 * **child_orphanage** - колличество детей нуждаются в устройстве в семьи (чел.);

 * **child_adopted** - доля детей устроенных в семьи от нуждающихся (чел.);

 * **pension** - размер пенсии (руб.);

 * **pensioners** - колличество пенсионеров региона (чел.);

 * **pensioners_percent** - доля пенсионеров от населения региона (%);

 * **living_wage** - прожиточный минимум региона (руб.);

 * **unemployment_rate** - уровень безработицы (%);

 * **newborn** - рождаемость за год (чел.);

 * **child_mortality_rural** - смертность детей в сельской местности на первом году жизни за год (чел.);

 * **child_mortality_urban** - смертность детей в городской местности на первом году жизни за год (чел.);

 * **child_mortality** - смертность детей на первом году жизни за год (чел.);

 * **morbidity** - число зарегистрированных заболеваний у пациентов с диагнозом, установленном впервые в жизни;

 * **respiratory** - число респираторных заболеваний (на 1 тыс. человек) за год;

 * **neoplasms** - число онкологических заболеваний (на 1 тыс. человек) за год;

 * **disabled_18_60** - число людей трудоспособного возраста с инвалидностью;

 * **density** - плотность населения региона;

 * **spec_mining** - удельный объем региональной добычи полезных искпаемых на душу населения (млн.руб.);

 * **spec_manufact** - удельный объем регионального обраьатывающего производства на душу населения (млн.руб.);

 * **spec_indust** - удельный объем регионального промышленного производства на душу населения (млн.руб.);

 * **spec_water_supply** - удельный водоснабжения, организации сбора и утилизации отходов  на душу населения (млн.руб.);

 * **spec_provision** - удельный объем обеспечения электроэнергией, газом и паром на душу населения (млн.руб.);

 * **spec_reg_prod** - удельный объем регионального производства на душу населения.

 
